# Kapitola 29: Vicevrstve neuronove site - Jak vyresit problem XOR

---

## Co se naucite

V teto kapitole se naucite:
- Proc jednoduchy **Perceptron selze** na problemu XOR
- Co je **vicevrstvy Perceptron (MLP)**
- Jak funguje **skryta vrstva**
- Princip **zpetneho sireni chyby (backpropagation)**
- Naprogramovat **MLP od nuly** pro XOR
- Pouzit **sklearn MLPClassifier** pro klasifikaci

## Proc vicevrstve site?

V minule kapitole jsme vytvorili Perceptron, ktery zvladl AND a OR.
Ale na XOR selhal - jednou primkou nelze oddelit diagonalni body.

**Reseni**: Vice neuronu ve vrstvach = schopnost ucit se slozitejsi vzory!

## 1. Instalace a import knihoven

In [ ]:
# Instalace knihoven
!pip install numpy matplotlib scikit-learn -q

print("Knihovny nainstalovany!")

In [ ]:
# Import knihoven
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons, make_circles

# Nastaveni
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

print("NumPy verze:", np.__version__)
print("Prostredi pripraveno!")

## 2. Problem XOR - Proc Perceptron selhava

XOR (exclusive or) vraci 1 pouze kdyz vstupy jsou **ruzne**:

| Vstup 1 | Vstup 2 | XOR |
|---------|---------|-----|
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

**Proc Perceptron selhava?** Nelze nakreslit jednu primku, ktera by oddělila body!

In [ ]:
# Vizualizace problemu XOR
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])

plt.figure(figsize=(10, 8))

# Body
colors = ['red' if y == 0 else 'blue' for y in y_xor]
plt.scatter(X_xor[:, 0], X_xor[:, 1], c=colors, s=300, edgecolors='black', linewidths=2, zorder=5)

# Popisky
labels = ['(0,0) = 0', '(0,1) = 1', '(1,0) = 1', '(1,1) = 0']
positions = [(0.12, 0.12), (-0.15, 1.12), (1.12, -0.12), (1.12, 1.12)]
for i, (label, pos) in enumerate(zip(labels, positions)):
    plt.annotate(label, xy=(X_xor[i, 0], X_xor[i, 1]), xytext=pos, fontsize=12, fontweight='bold')

# Neuspesne pokusy o oddeleni
x_line = np.linspace(-0.3, 1.3, 100)
plt.plot(x_line, 0.5 * np.ones_like(x_line), 'g--', alpha=0.5, linewidth=2, label='Horizontalni primka - NEFUNGUJE')
plt.plot(0.5 * np.ones_like(x_line), x_line, 'm--', alpha=0.5, linewidth=2, label='Vertikalni primka - NEFUNGUJE')
plt.plot(x_line, x_line, 'orange', linestyle='--', alpha=0.5, linewidth=2, label='Diagonalni primka - NEFUNGUJE')

plt.xlim(-0.3, 1.5)
plt.ylim(-0.3, 1.5)
plt.xlabel('Vstup 1', fontsize=12)
plt.ylabel('Vstup 2', fontsize=12)
plt.title('Problem XOR: Nelze oddelit jednou primkou!', fontsize=14, fontweight='bold')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("XOR je NELINEARNI problem - jeden neuron ho nezvladne!")
print("Potrebujeme VICEVRSTOU NEURONOVOU SIT (MLP).")

## 3. Reseni: Vicevrstvy Perceptron (MLP)

**Multi-Layer Perceptron (MLP)** pridava mezi vstup a vystup **skrytou vrstvu**.

```
VSTUPNI VRSTVA     SKRYTA VRSTVA     VYSTUPNI VRSTVA
   (2 neurony)      (2 neurony)        (1 neuron)
       
      x1  ─────┐     ┌───  h1  ───┐
               ├─────┤            ├───── y
      x2  ─────┘     └───  h2  ───┘
```

**Kazdy neuron ve skryte vrstve** se nauci rozpoznavat jiny vzor.
**Vystupni neuron** kombinuje jejich vystupy do finalniho rozhodnuti.

In [ ]:
# Vizualizace architektury MLP
fig, ax = plt.subplots(figsize=(14, 8))
ax.axis('off')

# Pozice neuronu
layer_positions = {
    'input': [(0.1, 0.7), (0.1, 0.3)],
    'hidden': [(0.5, 0.8), (0.5, 0.5), (0.5, 0.2)],
    'output': [(0.9, 0.5)]
}

# Vykreslit neurony
for layer, positions in layer_positions.items():
    for pos in positions:
        if layer == 'input':
            color = 'lightgreen'
        elif layer == 'hidden':
            color = 'lightblue'
        else:
            color = 'lightyellow'
        circle = plt.Circle(pos, 0.06, color=color, ec='black', lw=2)
        ax.add_patch(circle)

# Spojeni mezi vrstvami
for inp in layer_positions['input']:
    for hid in layer_positions['hidden']:
        ax.annotate('', xy=(hid[0]-0.06, hid[1]), xytext=(inp[0]+0.06, inp[1]),
                   arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))

for hid in layer_positions['hidden']:
    for out in layer_positions['output']:
        ax.annotate('', xy=(out[0]-0.06, out[1]), xytext=(hid[0]+0.06, hid[1]),
                   arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))

# Popisky
ax.text(0.1, 0.95, 'VSTUPNI VRSTVA', ha='center', fontsize=12, fontweight='bold', color='green')
ax.text(0.5, 0.95, 'SKRYTA VRSTVA', ha='center', fontsize=12, fontweight='bold', color='blue')
ax.text(0.9, 0.95, 'VYSTUPNI VRSTVA', ha='center', fontsize=12, fontweight='bold', color='orange')

ax.text(0.1, 0.7, 'x₁', ha='center', va='center', fontsize=14, fontweight='bold')
ax.text(0.1, 0.3, 'x₂', ha='center', va='center', fontsize=14, fontweight='bold')
ax.text(0.5, 0.8, 'h₁', ha='center', va='center', fontsize=14, fontweight='bold')
ax.text(0.5, 0.5, 'h₂', ha='center', va='center', fontsize=14, fontweight='bold')
ax.text(0.5, 0.2, 'h₃', ha='center', va='center', fontsize=14, fontweight='bold')
ax.text(0.9, 0.5, 'y', ha='center', va='center', fontsize=14, fontweight='bold')

# Poznamky
ax.text(0.5, 0.05, 'Kazdy neuron ve skryte vrstve se nauci rozpoznavat jiny vzor.\nVystupni neuron kombinuje jejich vystupy.', 
       ha='center', fontsize=11, style='italic')

plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('Architektura MLP: 2 vstupy - 3 skryte neurony - 1 vystup', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Aktivacni funkce Sigmoid

Misto skokove funkce pouzijeme **Sigmoid** - hladka krivka mezi 0 a 1.

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

**Proc Sigmoid?**
- Hladka derivace umoznuje uceni pomoci gradientu
- Vraci pravdepodobnost (hodnota mezi 0 a 1)

In [ ]:
# Sigmoid funkce a jeji derivace
def sigmoid(x):
    """Sigmoid aktivacni funkce"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))  # Clip pro numericou stabilitu

def sigmoid_derivative(x):
    """Derivace sigmoidu - potrebna pro backpropagation"""
    return x * (1 - x)

# Vizualizace
x = np.linspace(-6, 6, 100)
y_sig = sigmoid(x)
y_deriv = sigmoid(x) * (1 - sigmoid(x))

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(x, y_sig, 'b-', linewidth=3, label='σ(x)')
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.fill_between(x, 0, y_sig, alpha=0.2)
plt.title('Sigmoid funkce', fontweight='bold', fontsize=14)
plt.xlabel('x')
plt.ylabel('σ(x)')
plt.ylim(-0.1, 1.1)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(x, y_deriv, 'r-', linewidth=3, label="σ'(x)")
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.fill_between(x, 0, y_deriv, alpha=0.2, color='red')
plt.title('Derivace sigmoidu', fontweight='bold', fontsize=14)
plt.xlabel('x')
plt.ylabel("σ'(x)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Sigmoid: Hladky prechod mezi 0 a 1")
print("Derivace: Maximalni v bode 0, umoznuje gradientni uceni")

## 5. Backpropagation - Zpetne sireni chyby

**Backpropagation** je algoritmus pro uceni vicevrstvych siti:

1. **Forward pass**: Posli data siti a vypocti vystup
2. **Vypocti chybu**: Porovnej vystup s ocekavanou hodnotou
3. **Backward pass**: Propaguj chybu zpet a uprav vahy

**Analogie**: Ticha posta pozpatku s obviňovanim - kazda vrstva dostane svuj "dil viny" za chybu.

In [ ]:
# Vizualizace backpropagation
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Forward pass
ax1 = axes[0]
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')
ax1.set_title('1. FORWARD PASS', fontweight='bold', fontsize=12, color='green')

# Neurony
for pos in [(0.2, 0.7), (0.2, 0.3)]:
    ax1.add_patch(plt.Circle(pos, 0.08, color='lightgreen', ec='black', lw=2))
for pos in [(0.5, 0.7), (0.5, 0.3)]:
    ax1.add_patch(plt.Circle(pos, 0.08, color='lightblue', ec='black', lw=2))
ax1.add_patch(plt.Circle((0.8, 0.5), 0.08, color='lightyellow', ec='black', lw=2))

# Sipky doprava
for start in [(0.2, 0.7), (0.2, 0.3)]:
    for end in [(0.5, 0.7), (0.5, 0.3)]:
        ax1.annotate('', xy=(end[0]-0.08, end[1]), xytext=(start[0]+0.08, start[1]),
                    arrowprops=dict(arrowstyle='->', color='green', lw=2))
for start in [(0.5, 0.7), (0.5, 0.3)]:
    ax1.annotate('', xy=(0.72, 0.5), xytext=(start[0]+0.08, start[1]),
                arrowprops=dict(arrowstyle='->', color='green', lw=2))

ax1.text(0.5, 0.1, 'Data jdou DOPREDU\npres sit', ha='center', fontsize=10)

# Compute error
ax2 = axes[1]
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')
ax2.set_title('2. VYPOCET CHYBY', fontweight='bold', fontsize=12, color='red')

ax2.add_patch(plt.Circle((0.5, 0.6), 0.1, color='lightyellow', ec='black', lw=2))
ax2.text(0.5, 0.6, 'y', ha='center', va='center', fontsize=14, fontweight='bold')
ax2.text(0.5, 0.35, 'Chyba = y_skutecne - y_predikovane', ha='center', fontsize=11, color='red')
ax2.text(0.5, 0.2, 'E = (target - output)²', ha='center', fontsize=11)

# Backward pass
ax3 = axes[2]
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)
ax3.axis('off')
ax3.set_title('3. BACKWARD PASS', fontweight='bold', fontsize=12, color='blue')

# Neurony
for pos in [(0.2, 0.7), (0.2, 0.3)]:
    ax3.add_patch(plt.Circle(pos, 0.08, color='lightgreen', ec='black', lw=2))
for pos in [(0.5, 0.7), (0.5, 0.3)]:
    ax3.add_patch(plt.Circle(pos, 0.08, color='lightblue', ec='black', lw=2))
ax3.add_patch(plt.Circle((0.8, 0.5), 0.08, color='lightyellow', ec='black', lw=2))

# Sipky DOLEVA (zpetne sireni)
for end in [(0.5, 0.7), (0.5, 0.3)]:
    ax3.annotate('', xy=(end[0]+0.08, end[1]), xytext=(0.72, 0.5),
                arrowprops=dict(arrowstyle='->', color='red', lw=2))
for end in [(0.2, 0.7), (0.2, 0.3)]:
    for start in [(0.5, 0.7), (0.5, 0.3)]:
        ax3.annotate('', xy=(end[0]+0.08, end[1]), xytext=(start[0]-0.08, start[1]),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2))

ax3.text(0.5, 0.1, 'Chyba jde ZPET\na upravuje vahy', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## 6. Implementace MLP od nuly pro XOR

In [ ]:
class MLP_XOR:
    """
    Vicevrstvy Perceptron (MLP) pro problem XOR
    Architektura: 2 vstupy -> 2 skryte neurony -> 1 vystup
    """
    
    def __init__(self, learning_rate=0.5):
        self.learning_rate = learning_rate
        self.loss_history = []
        
        # Inicializace vah (nahodne hodnoty mezi -1 a 1)
        # Vahy mezi vstupni a skrytou vrstvou (2 vstupy -> 2 skryte neurony)
        self.weights_input_hidden = np.random.uniform(-1, 1, (2, 2))
        self.bias_hidden = np.random.uniform(-1, 1, (1, 2))
        
        # Vahy mezi skrytou a vystupni vrstvou (2 skryte -> 1 vystup)
        self.weights_hidden_output = np.random.uniform(-1, 1, (2, 1))
        self.bias_output = np.random.uniform(-1, 1, (1, 1))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, X):
        """
        Forward pass - vypocet vystupu site
        """
        # Skryta vrstva
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)
        
        # Vystupni vrstva
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = self.sigmoid(self.output_input)
        
        return self.predicted_output
    
    def backward(self, X, y):
        """
        Backward pass - zpetne sireni chyby a aktualizace vah
        """
        # Chyba na vystupu
        output_error = y - self.predicted_output
        output_delta = output_error * self.sigmoid_derivative(self.predicted_output)
        
        # Chyba ve skryte vrstve
        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)
        
        # Aktualizace vah (gradientni sestup)
        self.weights_hidden_output += self.hidden_output.T.dot(output_delta) * self.learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * self.learning_rate
        
        self.weights_input_hidden += X.T.dot(hidden_delta) * self.learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True) * self.learning_rate
        
        return np.mean(np.abs(output_error))
    
    def fit(self, X, y, epochs=10000, print_every=1000):
        """
        Trenovani site
        """
        y = y.reshape(-1, 1)  # Zajistit spravny tvar
        
        for epoch in range(epochs):
            # Forward pass
            self.forward(X)
            
            # Backward pass
            loss = self.backward(X, y)
            self.loss_history.append(loss)
            
            if (epoch + 1) % print_every == 0:
                print(f"Epocha {epoch + 1}/{epochs}, Chyba: {loss:.4f}")
        
        print("\nTrenovani dokonceno!")
        return self
    
    def predict(self, X):
        """
        Predikce (zaokrouhleno na 0 nebo 1)
        """
        output = self.forward(X)
        return np.round(output).astype(int).flatten()

print("Trida MLP_XOR definovana!")

In [ ]:
# Trenovani MLP na XOR
print("=" * 50)
print("TRENOVANI MLP NA PROBLEMU XOR")
print("=" * 50)

# Data
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])

# Vytvoreni a trenovani modelu
mlp = MLP_XOR(learning_rate=0.5)
mlp.fit(X_xor, y_xor, epochs=10000, print_every=2000)

In [ ]:
# Testovani modelu
print("\nVYSLEDKY:")
print("=" * 40)
print("Vstup    | Ocekavano | Predikovano | Spravne?")
print("-" * 40)

predictions = mlp.predict(X_xor)
raw_output = mlp.forward(X_xor)

for i in range(len(X_xor)):
    correct = "ANO" if predictions[i] == y_xor[i] else "NE"
    print(f"[{X_xor[i, 0]}, {X_xor[i, 1]}]   |     {y_xor[i]}     |      {predictions[i]}      |   {correct}")

print("-" * 40)
accuracy = np.mean(predictions == y_xor) * 100
print(f"Presnost: {accuracy:.0f}%")

print(f"\nSurove vystupy site (pred zaokrouhlenim):")
for i in range(len(X_xor)):
    print(f"  [{X_xor[i, 0]}, {X_xor[i, 1]}] -> {raw_output[i, 0]:.4f}")

In [ ]:
# Vizualizace uceni
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(mlp.loss_history, 'b-', linewidth=1)
plt.title('Pruběh učení MLP', fontweight='bold', fontsize=14)
plt.xlabel('Epocha')
plt.ylabel('Chyba (MAE)')
plt.grid(True, alpha=0.3)

# Rozhodovaci hranice
plt.subplot(1, 2, 2)

# Vytvoreni mrizky
xx, yy = np.meshgrid(np.linspace(-0.5, 1.5, 100), np.linspace(-0.5, 1.5, 100))
grid_points = np.c_[xx.ravel(), yy.ravel()]
Z = mlp.forward(grid_points)
Z = Z.reshape(xx.shape)

# Vykresleni
plt.contourf(xx, yy, Z, levels=50, cmap='RdYlBu', alpha=0.8)
plt.colorbar(label='Vystup site')
plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)

# Body
colors = ['red' if y == 0 else 'blue' for y in y_xor]
plt.scatter(X_xor[:, 0], X_xor[:, 1], c=colors, s=200, edgecolors='black', linewidths=2, zorder=5)

plt.title('Rozhodovaci hranice MLP pro XOR', fontweight='bold', fontsize=14)
plt.xlabel('Vstup 1')
plt.ylabel('Vstup 2')

plt.tight_layout()
plt.show()

print("MLP dokaze nareslit NELINEARNI rozhodovaci hranici!")
print("Tim vyresi problem XOR, ktery jednoduchy Perceptron nezvladne.")

## 7. MLPClassifier ze scikit-learn

V praxi pouzivame hotove implementace. Scikit-learn ma `MLPClassifier`.

In [ ]:
# MLPClassifier ze sklearn
from sklearn.neural_network import MLPClassifier

# Vytvoreni modelu
mlp_sklearn = MLPClassifier(
    hidden_layer_sizes=(4,),  # 1 skryta vrstva se 4 neurony
    activation='relu',         # Aktivacni funkce (relu je moderni)
    solver='adam',             # Optimalizator
    max_iter=1000,
    random_state=42
)

# Trenovani
mlp_sklearn.fit(X_xor, y_xor)

# Predikce
predictions_sklearn = mlp_sklearn.predict(X_xor)

print("MLPClassifier ze sklearn:")
print(f"Predikce: {predictions_sklearn}")
print(f"Ocekavano: {y_xor}")
print(f"Presnost: {mlp_sklearn.score(X_xor, y_xor) * 100:.0f}%")

## 8. Mini-projekt: Klasifikace slozitejsich dat

In [ ]:
# Generovani slozitejsich dat - "mesice"
X_moons, y_moons = make_moons(n_samples=200, noise=0.15, random_state=42)

print(f"Dataset 'Moons': {len(X_moons)} vzorku")

# Vizualizace
plt.figure(figsize=(8, 6))
plt.scatter(X_moons[y_moons==0, 0], X_moons[y_moons==0, 1], c='red', label='Trida 0', alpha=0.6)
plt.scatter(X_moons[y_moons==1, 0], X_moons[y_moons==1, 1], c='blue', label='Trida 1', alpha=0.6)
plt.title('Dataset "Moons" - nelinearni problem', fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Porovnani ruznych architektur MLP
from sklearn.model_selection import train_test_split

# Rozdeleni dat
X_train, X_test, y_train, y_test = train_test_split(
    X_moons, y_moons, test_size=0.3, random_state=42
)

# Ruzne architektury
architektury = [
    (2,),           # 1 vrstva, 2 neurony
    (10,),          # 1 vrstva, 10 neuronu
    (10, 5),        # 2 vrstvy: 10 a 5 neuronu
    (20, 10, 5),    # 3 vrstvy
]

print("Porovnani architektur MLP na datasetu 'Moons':")
print("=" * 55)
print(f"{'Architektura':<20} | {'Train Acc':<10} | {'Test Acc':<10}")
print("-" * 55)

modely = []
for arch in architektury:
    model = MLPClassifier(
        hidden_layer_sizes=arch,
        activation='relu',
        solver='adam',
        max_iter=1000,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    train_acc = model.score(X_train, y_train) * 100
    test_acc = model.score(X_test, y_test) * 100
    
    print(f"{str(arch):<20} | {train_acc:>8.1f}% | {test_acc:>8.1f}%")
    modely.append((arch, model, test_acc))

In [ ]:
# Vizualizace rozhodovacich hranic pro ruzne architektury
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for idx, (arch, model, acc) in enumerate(modely):
    ax = axes.flatten()[idx]
    
    # Mrizka
    xx, yy = np.meshgrid(
        np.linspace(X_moons[:, 0].min()-0.5, X_moons[:, 0].max()+0.5, 100),
        np.linspace(X_moons[:, 1].min()-0.5, X_moons[:, 1].max()+0.5, 100)
    )
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Vykresleni
    ax.contourf(xx, yy, Z, alpha=0.4, cmap='RdYlBu')
    ax.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
    ax.scatter(X_moons[y_moons==0, 0], X_moons[y_moons==0, 1], c='red', s=20, alpha=0.6)
    ax.scatter(X_moons[y_moons==1, 0], X_moons[y_moons==1, 1], c='blue', s=20, alpha=0.6)
    
    ax.set_title(f'Architektura: {arch}\nTest presnost: {acc:.1f}%', fontweight='bold')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

plt.suptitle('Vliv architektury MLP na rozhodovaci hranici', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Vice neuronu = slozitejsi rozhodovaci hranice")
print("Ale pozor na PREUCENI (overfitting)!")

## 9. Shrnuti a klicove poznatky

### Co jsme se naucili:

| Koncept | Popis |
|---------|-------|
| **MLP** | Vicevrstvy Perceptron - site s 1+ skrytymi vrstvami |
| **Skryta vrstva** | Neurony mezi vstupem a vystupem |
| **Sigmoid** | Hladka aktivacni funkce (0-1) |
| **Backpropagation** | Algoritmus pro uceni - zpetne sireni chyby |
| **XOR problem** | Nelinearni problem, ktery MLP zvladne |

### Klicove poznatky:

1. **Perceptron je omezeny** na linearni problemy
2. **MLP pridava skryte vrstvy** pro uceni nelinearnich vzoru
3. **Backpropagation** umoznuje uceni hlubokych siti
4. **Vice neuronu** = schopnost ucit se slozitejsi vzory
5. **Pozor na overfitting** - prilis velka sit muze preucit data

## 10. Kviz: Otestujte sve znalosti

In [ ]:
def kviz_mlp():
    print("=" * 60)
    print("KVIZ: Vicevrstve neuronove site")
    print("=" * 60)
    
    otazky = [
        {
            "otazka": "Proc jednoduchy Perceptron nemuze vyresit XOR?",
            "moznosti": ["a) Ma malo vah", "b) XOR neni linearni separovatelny", "c) Chybi mu bias", "d) Je prilis rychly"],
            "spravne": "b",
            "vysvetleni": "XOR nelze oddelit jednou primkou, proto Perceptron selhava."
        },
        {
            "otazka": "Co je skryta vrstva v MLP?",
            "moznosti": ["a) Vrstva, ktera je neviditelna", "b) Vrstva mezi vstupem a vystupem", "c) Vystupni vrstva", "d) Vstupni vrstva"],
            "spravne": "b",
            "vysvetleni": "Skryta vrstva je mezi vstupni a vystupni vrstvou a uci se mezivystupy."
        },
        {
            "otazka": "Co je backpropagation?",
            "moznosti": ["a) Smer toku dat", "b) Algoritmus pro zpetne sireni chyby", "c) Typ aktivacni funkce", "d) Zpusob generovani dat"],
            "spravne": "b",
            "vysvetleni": "Backpropagation siri chybu zpet siti a upravuje vahy."
        },
        {
            "otazka": "Proc pouzivame sigmoid misto skokove funkce?",
            "moznosti": ["a) Je rychlejsi", "b) Ma hladkou derivaci pro gradientni uceni", "c) Vraci vetsi hodnoty", "d) Je jednodussi"],
            "spravne": "b",
            "vysvetleni": "Sigmoid ma hladkou derivaci, ktera umoznuje gradientni sestup."
        },
        {
            "otazka": "Co se stane kdyz pouzijeme prilis velkou sit?",
            "moznosti": ["a) Vzdy se zlepsi presnost", "b) Muze dojit k overfittingu", "c) Nic se nedeje", "d) Sit se neda natrenovat"],
            "spravne": "b",
            "vysvetleni": "Prilis velka sit se muze preucit na trenovacich datech."
        }
    ]
    
    skore = 0
    
    for i, q in enumerate(otazky, 1):
        print(f"\n{i}. {q['otazka']}")
        for m in q['moznosti']:
            print(f"   {m}")
        
        odpoved = input("   Vase odpoved (a/b/c/d): ").strip().lower()
        
        if odpoved == q['spravne']:
            print("   Spravne!")
            skore += 1
        else:
            print(f"   Spatne. Spravna odpoved je {q['spravne']})")
        print(f"   Vysvetleni: {q['vysvetleni']}")
    
    print("\n" + "=" * 60)
    print(f"VYSLEDEK: {skore}/{len(otazky)} spravnych odpovedi")

# Spustit kviz
kviz_mlp()

## 11. Vyzva pro vas

1. **Experimentujte s learning rate**: Zkuste 0.1, 0.01, 1.0 - jak to ovlivni uceni?
2. **Zmente pocet epoch**: Co se stane pri 1000, 50000?
3. **Circles dataset**: Pouzijte `make_circles()` a zkuste ruzne architektury

In [ ]:
# Prostor pro experimenty
# X_circles, y_circles = make_circles(n_samples=200, noise=0.1, factor=0.5, random_state=42)



---

## Zaver

V teto kapitole jste:

- Pochopili **limitace jednoducheho Perceptronu**
- Naucili se, co je **vicevrstvy Perceptron (MLP)**
- Implementovali **MLP od nuly** pro problem XOR
- Pochopili princip **backpropagation**
- Pouzili **sklearn MLPClassifier**

**MLP je zakladem modernich neuronovych siti!** GPT, obrazove modely - vse stoji na techto principech.

---
*Kapitola 29 | Vicevrstve neuronove site | AI Akademie*